# Setup, Pre-processing

In [ ]:
# !pip install gurobipy
import gurobipy as gb
from gurobipy import *
import pandas as pd
import numpy as np

In [ ]:
ev_data = pd.read_csv("WA_Electric_Vehicle_Population_Data.csv", dtype={'Postal Code': str})
distance_data = pd.read_csv("distance.csv", dtype={'PostalCode1': str, 'PostalCode2': str})
# ev_data = pd.read_csv("/content/Electric_Vehicle_Population_Data.csv", dtype={'Postal Code': str})
# distance_data = pd.read_csv("/content/distance.csv", dtype={'PostalCode1': str, 'PostalCode2': str})

In [ ]:
# Define coverage
def calculate_coverage(row, average_ranges, theta):
    postal1 = row['PostalCode1']
    postal2 = row['PostalCode2']
    distance = row['Distance']

    coverage1 = 1 if (theta * average_ranges.get(postal1, 0) >= distance) else 0
    coverage2 = 1 if (theta * average_ranges.get(postal2, 0) >= distance) else 0

    return pd.Series([coverage1, coverage2])

def calculate_C(distance_data):
    # C(i, j) = 1 if i is covered by j, otherwise C(i, j) = 0
    C = {}
    for _, row in distance_data.iterrows():
        C[(row['PostalCode1'], row['PostalCode2'])] = row['Coverage1']
        C[(row['PostalCode2'], row['PostalCode1'])] = row['Coverage2']

    # Ensure each location covers itself
    postal_codes = set(distance_data['PostalCode1']).union(distance_data['PostalCode2'])
    for postal_code in postal_codes:
        C[(postal_code, postal_code)] = 1

    return C

# Calculate the count of EVs in each zip code
EV_count = ev_data['Postal Code'].value_counts().to_dict()

# Calculate the average electric range for each zip code
average_range = ev_data.groupby('Postal Code')['Electric Range'].mean().to_dict()

# Calculate demand (EV_j) as the total EV range in each zip code
EV = {key: EV_count[key] * average_range[key] for key in EV_count if key in average_range}

distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
    calculate_coverage,
    axis=1,
    average_ranges=average_range,
    theta=0.2
)
C = calculate_C(distance_data)

# Base

In [ ]:
############################### Initialize Model ###############################

model = gb.Model("EV_Charging_Optimization")
model.setParam('MIPGap', 1e-05)
model.Params.OutputFlag = 0


################################## Parameters ##################################

# Charging speeds in miles per hour
charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
environmental_impact = {2: 0.000027405, 3: 0.0002945}
H = 10  # Available hours per day
public_charger_demand = 0.3
avg_percentage_charged = 0.6
peak_charging_ev = 0.1

# Cost parameters
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
# Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
base_station_cost = 50000
# Electrical upgrade calculations
panel_capacity = 500 # amps
usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
existing_load = 200  # amps
remaining_capacity = usable_capacity - existing_load  # amps
upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers


############################## Decision Variables ##############################

x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")

demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")


################################## Objectives ##################################

total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)

# Calculate the environmental impact based on demand fulfilled by each type
total_environmental_impact = (
    environmental_impact[2] * demand_fulfilled_by_level2 +
    environmental_impact[3] * demand_fulfilled_by_level3
)
#### Calculate the total cost
M = 10000
for i in EV:
    # Ensure only one tier is active for each location
    model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

    # Combined loads:
    combined_loads = x2[i] * 40 + x3[i] * 150

    # Constraints for each tier
    model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")

electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
chargers_setup_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
)
total_cost = chargers_setup_cost + station_setup_cost

# 1. Minimize the total cost
model.setObjectiveN(total_cost, index=0, priority=0)

# 2. Minimize Environmental impact
model.setObjectiveN(total_environmental_impact, index=1, priority=1)

# 3. Maximize total electric range per hour of charging
model.setObjectiveN(-total_range, index=2, priority=2)

model.ModelSense = GRB.MINIMIZE


################################## Constraints #################################
# EV at any location can reach at least one of the charging stations
for j in EV:
    model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

# Demand Coverage with Charging Capacity Requirement
for j in EV:
    total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
    model.addConstr(
        gb.quicksum(
            C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
            for i in EV
        ) >= total_demand_j,
        name=f"DemandCoverage_{j}"
    )

# Calculate the demand fulfilled by each type of charger
model.addConstr(
    demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
    name="DemandFulfilledByLevel2"
)
model.addConstr(
    demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
    name="DemandFulfilledByLevel3"
)

# # Simultaneous charging
# model.addConstr(gb.quicksum((x2[i] + x3[i]) for i in EV) / (sum(EV_count[j] for j in EV) * public_charger_demand) >= peak_charging_ev)
# for j in EV:
#     model.addConstr(gb.quicksum(C.get((i, j), 0) * (x2[i] + x3[i]) * z[i] for i in EV) >= EV_count[j] * public_charger_demand * peak_charging_ev)

# Linking Charging Stations and Chargers
M = 20  # Maximum number of charging stalls at each station
for i in EV:
    model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
    model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)


################################# Solve Model ##################################

model.optimize()

if model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Charging Capacity per Hour: {total_range.getValue()}")
    print(f'Total Environmental Impact: {total_environmental_impact.getValue()}')
    print(f'Total Cost: {total_cost.getValue()}')
    print(f"Total Demand Fulfilled by Level 2 Chargers (miles): {demand_fulfilled_by_level2.x}")
    print(f"Total Demand Fulfilled by Level 3 Chargers (miles): {demand_fulfilled_by_level3.x}")
    print(f'Total number of Level 2 Charger: {sum(x2[i].x for i in EV)}')
    print(f'Total number of Level 3 Charger: {sum(x3[i].x for i in EV)}')
    print(f'Total number of Station: {sum(z[i].x for i in EV)}')
    print("\n")
    for i in EV:
        if z[i].x > 0.5:  # Using 0.5 to account for numerical issues
            level2_chargers = int(x2[i].x)
            level3_chargers = int(x3[i].x)
            print(f"Set up station at {i} with {level2_chargers} Level 2 chargers and {level3_chargers} Level 3 chargers")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2026-11-23
Set parameter MIPGap to value 1e-05


GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

## Save Results

In [ ]:
data = []
col = ['PostalCode', 'StationSetup', 'Level2Chargers', 'Level3Chargers']

for i in EV:
    if z[i].x > 0.5:
        level2_chargers = int(round(x2[i].x))
        level3_chargers = int(round(x3[i].x))
        data.append([i, 1, level2_chargers, level3_chargers])
    else:
        data.append([i, 0, 0, 0])

df = pd.DataFrame(data, columns=col)
df.to_csv('results.csv', index=False)

from google.colab import files
files.download('results.csv')

# Extension 1: Parameter Sensitivity

Explore multiple values for threshold to understand how different assumptions affect station coverage and the total range. A sensitivity analysis on theta can help us find an optimal balance.

## Vary Coverage Threshold

In [ ]:
thetas = [0.1, 0.2, 0.3, 0.4, 0.5]
results = []

for theta in thetas:
    distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
        calculate_coverage,
        axis=1,
        average_ranges=average_range,
        theta=theta
    )
    C = calculate_C(distance_data)

    ########## Initialize Model ##########
    model = gb.Model(f"Extension 1: coverage threshold = {theta}")
    model.setParam('MIPGap', 1e-05)
    model.Params.OutputFlag = 0

    ########## Parameters ##########
    # Charging speeds in miles per hour
    charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
    environmental_impact = {2: 0.000027405, 3: 0.0002945}
    H = 10  # Available hours per day
    public_charger_demand = 0.3
    avg_percentage_charged = 0.6
    # Cost parameters (charger setup and station setup)
    cost_per_level2_charger = 8250
    cost_per_level3_charger = 65000
    # Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
    base_station_cost = 50000
    # Electrical upgrade calculations
    panel_capacity = 500 # amps
    usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
    existing_load = 200  # amps
    remaining_capacity = usable_capacity - existing_load  # amps
    upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers

    ########## Decision Variables ##########
    # Number of Level 2 and Level 3 chargers at each station
    x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
    x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
    z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
    y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")  # Binary variables for electrical upgrade tiers

    # Demand fulfilled by each charger type in miles
    demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
    demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")

    ########## Objectives ##########
    total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
    total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
    total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)
    # Calculate the environmental impact based on demand fulfilled by each type
    total_environmental_impact = (
        environmental_impact[2] * demand_fulfilled_by_level2 +
        environmental_impact[3] * demand_fulfilled_by_level3
    )
    #### Calculate the total cost
    M = 10000
    for i in EV:
        # Ensure only one tier is active for each location
        model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

        # Combined loads:
        combined_loads = x2[i] * 40 + x3[i] * 150

        # Constraints for each tier
        model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")

    electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

    station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
    chargers_setup_cost = gb.quicksum(
        x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
    )
    total_cost = chargers_setup_cost + station_setup_cost

    # 1. Minimize the total cost
    model.setObjectiveN(total_cost, index=0, priority=0)

    # 2. Minimize Environmental impact
    model.setObjectiveN(total_environmental_impact, index=1, priority=1)

    # 3. Maximize total electric range per hour of charging
    model.setObjectiveN(-total_range, index=2, priority=2)

    model.ModelSense = GRB.MINIMIZE

    ########## Constraints ##########
    # EV at any location can reach at least one of the charging stations
    for j in EV:
        model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

    # Demand Coverage with Charging Capacity Requirement
    for j in EV:
        total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
        model.addConstr(
            gb.quicksum(
                C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
                for i in EV
            ) >= total_demand_j,
            name=f"DemandCoverage_{j}"
        )

    # Calculate the demand fulfilled by each type of charger
    model.addConstr(
        demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
        name="DemandFulfilledByLevel2"
    )
    model.addConstr(
        demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
        name="DemandFulfilledByLevel3"
    )

    # Linking Charging Stations and Chargers
    M = 20  # Maximum number of charging stalls at each station
    for i in EV:
        model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
        model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

    model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
    model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
    model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)

    ########## Solve Model ##########
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        results.append({
            'theta': theta,
            'Total Charging Capacity per Hour': total_range.getValue(),
            'Total Environmental Impact': total_environmental_impact.getValue(),
            'Total Cost': total_cost.getValue(),
            'demand_fulfilled_level2': demand_fulfilled_by_level2.x,
            'demand_fulfilled_level3': demand_fulfilled_by_level3.x,
            'Total number of Level 2 Charger': sum(x2[i].x for i in EV),
            'Total number of Level 3 Charger': sum(x3[i].x for i in EV),
            'Total number of Station': sum(z[i].x for i in EV)
        })
    else:
        print("No optimal solution found.")

pd.DataFrame(results)

## Vary Average Percentage Charged

In [ ]:
demand_scaling_factors = [0.5, 0.8, 1.0, 1.2, 1.5]
results = []

for scale in demand_scaling_factors:
    distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
        calculate_coverage,
        axis=1,
        average_ranges=average_range,
        theta=0.2
    )
    C = calculate_C(distance_data)

    ########## Initialize Model ##########
    model = gb.Model(f"Extension 1: percentage charged scaling = {scale}")
    model.setParam('MIPGap', 1e-05)
    model.Params.OutputFlag = 0

    ########## Parameters ##########
    # Charging speeds in miles per hour
    charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
    environmental_impact = {2: 0.000027405, 3: 0.0002945}
    H = 10  # Available hours per day
    public_charger_demand = 0.3
    avg_percentage_charged = 0.6 * scale
    # Cost parameters (charger setup and station setup)
    cost_per_level2_charger = 8250
    cost_per_level3_charger = 65000
    # Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
    base_station_cost = 50000
    # Electrical upgrade calculations
    panel_capacity = 500 # amps
    usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
    existing_load = 200  # amps
    remaining_capacity = usable_capacity - existing_load  # amps
    upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers

    ########## Decision Variables ##########
    # Number of Level 2 and Level 3 chargers at each station
    x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
    x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
    z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
    y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")  # Binary variables for electrical upgrade tiers

    # Demand fulfilled by each charger type in miles
    demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
    demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")

    ########## Objectives ##########
    total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
    total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
    total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)
    # Calculate the environmental impact based on demand fulfilled by each type
    total_environmental_impact = (
        environmental_impact[2] * demand_fulfilled_by_level2 +
        environmental_impact[3] * demand_fulfilled_by_level3
    )
    #### Calculate the total cost
    M = 10000
    for i in EV:
        # Ensure only one tier is active for each location
        model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

        # Combined loads:
        combined_loads = x2[i] * 40 + x3[i] * 150

        # Constraints for each tier
        model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
        model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
        model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")


    electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

    station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
    chargers_setup_cost = gb.quicksum(
        x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
    )
    total_cost = chargers_setup_cost + station_setup_cost

    # 1. Minimize the total cost
    model.setObjectiveN(total_cost, index=0, priority=0)

    # 2. Minimize Environmental impact
    model.setObjectiveN(total_environmental_impact, index=1, priority=1)

    # 3. Maximize total electric range per hour of charging
    model.setObjectiveN(-total_range, index=2, priority=2)

    model.ModelSense = GRB.MINIMIZE

    ########## Constraints ##########
    # EV at any location can reach at least one of the charging stations
    for j in EV:
        model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

    # Demand Coverage with Charging Capacity Requirement
    for j in EV:
        total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
        model.addConstr(
            gb.quicksum(
                C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
                for i in EV
            ) >= total_demand_j,
            name=f"DemandCoverage_{j}"
        )

    # Calculate the demand fulfilled by each type of charger
    model.addConstr(
        demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
        name="DemandFulfilledByLevel2"
    )
    model.addConstr(
        demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
        name="DemandFulfilledByLevel3"
    )

    # Linking Charging Stations and Chargers
    M = 20  # Maximum number of charging stalls at each station
    for i in EV:
        model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
        model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

    model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
    model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
    model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)

    ########## Solve Model ##########
    model.optimize()

    if model.Status == GRB.OPTIMAL:
        results.append({
            'Avg Percentage Charged': avg_percentage_charged,
            'Total Charging Capacity per Hour': total_range.getValue(),
            'Total Environmental Impact': total_environmental_impact.getValue(),
            'Total Cost': total_cost.getValue(),
            'demand_fulfilled_level2': demand_fulfilled_by_level2.x,
            'demand_fulfilled_level3': demand_fulfilled_by_level3.x,
            'Total number of Level 2 Charger': sum(x2[i].x for i in EV),
            'Total number of Level 3 Charger': sum(x3[i].x for i in EV),
            'Total number of Station': sum(z[i].x for i in EV)
        })
    else:
        print("No optimal solution found.")

pd.DataFrame(results)

# Extension 2: Weighted Multi-Objective Function

We’ll replace the priority-based objective setup with a single objective function where each component has a weight. This allows more flexibility in balancing environmental impact, total charging capacity, and costs.

In [ ]:
########## Initialize Model ##########
model = gb.Model("Extension 2: Weighted Multi-Objective Function")
model.setParam('MIPGap', 1e-05)
model.Params.OutputFlag = 0

########## Parameters ##########
# Charging speeds in miles per hour
charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
environmental_impact = {2: 0.000027405, 3: 0.0002945}
H = 10  # Available hours per day
public_charger_demand = 0.3
avg_percentage_charged = 0.6
# Cost parameters (charger setup and station setup)
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
# Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
base_station_cost = 50000
# Electrical upgrade calculations
panel_capacity = 500 # amps
usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
existing_load = 200  # amps
remaining_capacity = usable_capacity - existing_load  # amps
upgrade_costs = [0, 30000, 75000, 100000]  # Costs for tiers

########## Decision Variables ##########
# Number of Level 2 and Level 3 chargers at each station
x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")  # Binary variables for electrical upgrade tiers

# Demand fulfilled by each charger type in miles
demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")

########## Constraints ##########
# EV at any location can reach at least one of the charging stations
for j in EV:
    model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

# Demand Coverage with Charging Capacity Requirement
for j in EV:
    total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
    model.addConstr(
        gb.quicksum(
            C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
            for i in EV
        ) >= total_demand_j,
        name=f"DemandCoverage_{j}"
    )

# Calculate the demand fulfilled by each type of charger
model.addConstr(
    demand_fulfilled_by_level2 == charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
    name="DemandFulfilledByLevel2"
)
model.addConstr(
    demand_fulfilled_by_level3 == charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
    name="DemandFulfilledByLevel3"
)

# Linking Charging Stations and Chargers
M = 20  # Maximum number of charging stalls at each station
for i in EV:
    model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
    model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)

########## Objectives ##########
total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
total_range = gb.quicksum((x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) for i in EV)
# Calculate the environmental impact based on demand fulfilled by each type
total_environmental_impact = (
    environmental_impact[2] * demand_fulfilled_by_level2 +
    environmental_impact[3] * demand_fulfilled_by_level3
)
#### Calculate the total cost
M = 10000
for i in EV:
    # Ensure only one tier is active for each location
    model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

    # Combined loads:
    combined_loads = x2[i] * 40 + x3[i] * 150

    # Constraints for each tier
    model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")


electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
chargers_setup_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
)
total_cost = chargers_setup_cost + station_setup_cost

########## Weighted Objective Function ##########
# Define weights for each component of the objective function
# These weights should be chosen based on the relative importance of each objective
# For example purposes, arbitrary weights are assigned. Adjust these as needed.
weight_total_range = 150000      # Prioritize range
weight_environmental_impact = 10000  # Moderate weight for sustainability
weight_total_cost = 0.000001     # Small weight, ensures cost isn't excessive

# Formulate the single objective function
single_objective = (
    weight_total_range * (-total_range) +              # Maximize total range
    weight_environmental_impact * total_environmental_impact +  # Minimize environmental impact
    weight_total_cost * total_cost                      # Minimize total cost
)

model.setObjective(single_objective, GRB.MINIMIZE)

########## Solve Model ##########
model.optimize()

if model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Charging Capacity per Hour: {total_range.getValue()}")
    print(f'Total Environmental Impact: {total_environmental_impact.getValue()}')
    print(f'Total Cost: {total_cost.getValue()}')
    print(f"Total Demand Fulfilled by Level 2 Chargers (miles): {demand_fulfilled_by_level2.x}")
    print(f"Total Demand Fulfilled by Level 3 Chargers (miles): {demand_fulfilled_by_level3.x}")
    print(f'Total number of Level 2 Charger: {sum(x2[i].x for i in EV)}')
    print(f'Total number of Level 3 Charger: {sum(x3[i].x for i in EV)}')
    print(f'Total number of Station: {sum(z[i].x for i in EV)}')
    print("\n")
    for i in EV:
        if z[i].x > 0.5:  # Using 0.5 to account for numerical issues
            level2_chargers = int(x2[i].x)
            level3_chargers = int(x3[i].x)
            print(f"Set up station at {i} with {level2_chargers} Level 2 chargers and {level3_chargers} Level 3 chargers")
else:
    print("No optimal solution found.")

# Extension 3: Charger Breakdowns

In real life, chargers do not work a 100% of the times. Our original model assumes a 100% reliability of chargers. To make it more realistic, we will introduce a new factor called the reliability factor that would give us how reliable a certain type of charger is.

In [ ]:
# Initialize Model
model = gb.Model("EV_Charging_Optimization_Reliability")
model.setParam('MIPGap', 1e-05)
model.Params.OutputFlag = 0


################################## Parameters ##################################

# Charging speeds in miles per hour
charging_speed = {2: 15, 3: 210}  # Level 2 and Level 3 chargers
reliability_factor = {2: 0.95, 3: 0.90}
effective_charging_speed = {
    level: charging_speed[level] * reliability_factor[level] for level in charging_speed
}

environmental_impact = {2: 0.000027405, 3: 0.0002945}
H = 10  # Available hours per day
public_charger_demand = 0.3
avg_percentage_charged = 0.6
peak_charging_ev = 0.1

# Cost parameters
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
# Fixed costs (site preparation, permit, and others (station design, lighting, security systems))
base_station_cost = 50000
# Electrical upgrade calculations
panel_capacity = 500 # amps
usable_capacity = panel_capacity * 0.8  # general rule is to use up to 80% of total capacity
existing_load = 200  # amps
remaining_capacity = usable_capacity - existing_load  # amps
upgrade_costs = [0, 30000, 75000, 10000]  # Costs for tiers


############################## Decision Variables ##############################

x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
z = model.addVars(EV.keys(), vtype=GRB.BINARY, name="Station")  # Whether to set up a station
y = model.addVars(EV, range(4), vtype=GRB.BINARY, name="UpgradeTier")

demand_fulfilled_by_level2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel2")
demand_fulfilled_by_level3 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="DemandFulfilledLevel3")


################################## Objectives ##################################

total_demand = sum(EV.values()) * public_charger_demand * avg_percentage_charged
total_chargers = gb.quicksum(x2[i] + x3[i] for i in EV)
total_range = gb.quicksum((x2[i] * effective_charging_speed[2] + x3[i] * effective_charging_speed[3]) for i in EV)

# Calculate the environmental impact based on demand fulfilled by each type
total_environmental_impact = (
    environmental_impact[2] * demand_fulfilled_by_level2 +
    environmental_impact[3] * demand_fulfilled_by_level3
)
#### Calculate the total cost
M = 10000
for i in EV:
    # Ensure only one tier is active for each location
    model.addConstr(gb.quicksum(y[i, k] for k in range(4)) == 1, name=f"OnlyOneTierActive_{i}")

    # Combined loads:
    combined_loads = x2[i] * 40 + x3[i] * 150

    # Constraints for each tier
    model.addConstr(combined_loads <= remaining_capacity + M * (1 - y[i, 0]), name=f"NoUpgrade_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 1 - M * (1 - y[i, 1]), name=f"Upgrade1_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 300 + M * (1 - y[i, 1]), name=f"Upgrade1_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 301 - M * (1 - y[i, 2]), name=f"Upgrade2_Lower_{i}")
    model.addConstr(combined_loads <= remaining_capacity + 750 + M * (1 - y[i, 2]), name=f"Upgrade2_Upper_{i}")
    model.addConstr(combined_loads >= remaining_capacity + 751 - M * (1 - y[i, 3]), name=f"Upgrade3_{i}")

electrical_upgrade_cost = gb.quicksum(upgrade_costs[k] * y[i, k] for i in EV for k in range(4))

station_setup_cost = electrical_upgrade_cost + base_station_cost*gb.quicksum(z[i] for i in EV)
chargers_setup_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
)
total_cost = chargers_setup_cost + station_setup_cost

# 1. Minimize the total cost
model.setObjectiveN(total_cost, index=0, priority=0)

# 2. Minimize Environmental impact
model.setObjectiveN(total_environmental_impact, index=1, priority=1)

# 3. Maximize total electric range per hour of charging
model.setObjectiveN(-total_range, index=2, priority=2)

model.ModelSense = GRB.MINIMIZE


################################## Constraints #################################
# EV at any location can reach at least one of the charging stations
for j in EV:
    model.addConstr(gb.quicksum(C.get((i, j), 0) * z[i] for i in EV) >= 1)

# Demand Coverage with Charging Capacity Requirement
for j in EV:
    total_demand_j = EV[j] * public_charger_demand * avg_percentage_charged
    model.addConstr(
        gb.quicksum(
            C.get((i, j), 0) * (x2[i] * charging_speed[2] + x3[i] * charging_speed[3]) * H
            for i in EV
        ) >= total_demand_j,
        name=f"DemandCoverage_{j}"
    )

# Calculate the demand fulfilled by each type of charger
model.addConstr(
    demand_fulfilled_by_level2 == effective_charging_speed[2] * H * gb.quicksum(z[i] * x2[i] for i in EV),
    name="DemandFulfilledByLevel2"
)
model.addConstr(
    demand_fulfilled_by_level3 == effective_charging_speed[3] * H * gb.quicksum(z[i] * x3[i] for i in EV),
    name="DemandFulfilledByLevel3"
)

# Linking Charging Stations and Chargers
M = 20  # Maximum number of charging stalls at each station
for i in EV:
    model.addConstr(x2[i] + x3[i] <= M * z[i], name=f"MaxChargers_{i}")  # Chargers only if station is open
    model.addConstr(x2[i] + x3[i] >= z[i], name=f"MinChargers_{i}")  # At least one charger if station is open

model.addConstr(gb.quicksum(x2[i] for i in EV) <= 4546)
model.addConstr(gb.quicksum(x3[i] for i in EV) <= 1295)
model.addConstr(demand_fulfilled_by_level2 + demand_fulfilled_by_level3 <= 1.2 * total_demand)


# Solve the Model
model.optimize()

# Output Results
if model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Charging Capacity per Hour: {total_range.getValue()}")
    print(f'Total Environmental Impact: {total_environmental_impact.getValue()}')
    print(f'Total Cost: {total_cost.getValue()}')
    print(f"Total Demand Fulfilled by Level 2 Chargers (miles): {demand_fulfilled_by_level2.x}")
    print(f"Total Demand Fulfilled by Level 3 Chargers (miles): {demand_fulfilled_by_level3.x}")
    print(f'Total number of Level 2 Charger: {sum(x2[i].x for i in EV)}')
    print(f'Total number of Level 3 Charger: {sum(x3[i].x for i in EV)}')
    print(f'Total number of Station: {sum(z[i].x for i in EV)}')
    print("\n")
    for i in EV:
        if z[i].x > 0.5:  # Using 0.5 to account for numerical issues
            level2_chargers = int(x2[i].x)
            level3_chargers = int(x3[i].x)
            print(f"Set up station at {i} with {level2_chargers} Level 2 chargers and {level3_chargers} Level 3 chargers")
else:
    print("No optimal solution found.")


Set parameter MIPGap to value 1e-05
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
MIPGap  1e-05



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information

# Extension 4: Traveling Maintenance Man

Someone needs to maintain these chargers. A maintaince man is hired to travel between the stations and repair them.




In [ ]:
distance_data = pd.read_csv('/distance.csv')

# Extract the first 10 unique zipcodes
zipcodes = list(distance_data['PostalCode1'].unique()[:10])

# Create a dictionary to represent the graph
graph = {}
for zipcode in zipcodes:
    graph[zipcode] = {}

# Populate the graph with distances, converting to integers
for index, row in distance_data.iterrows():
    zip1 = row['PostalCode1']
    zip2 = row['PostalCode2']
    distance = int(row['Distance']) # Convert distance to integer

    if zip1 in zipcodes and zip2 in zipcodes:
        graph[zip1][zip2] = distance
        graph[zip2][zip1] = distance # Assuming distance is bidirectional

graph

{98122.0: {98109.0: 3,
  98027.0: 20,
  98597.0: 72,
  98903.0: 169,
  98012.0: 23,
  98366.0: 24,
  98001.0: 28,
  98144.0: 2,
  98110.0: 15},
 98109.0: {98122.0: 3,
  98027.0: 24,
  98597.0: 73,
  98903.0: 172,
  98012.0: 22,
  98366.0: 22,
  98001.0: 30,
  98144.0: 5,
  98110.0: 12},
 98027.0: {98122.0: 20,
  98109.0: 24,
  98597.0: 73,
  98903.0: 149,
  98012.0: 31,
  98366.0: 42,
  98001.0: 26,
  98144.0: 19,
  98110.0: 35},
 98597.0: {98122.0: 72,
  98109.0: 73,
  98027.0: 73,
  98903.0: 156,
  98012.0: 95,
  98366.0: 61,
  98001.0: 47,
  98144.0: 69,
  98110.0: 71},
 98903.0: {98122.0: 169,
  98109.0: 172,
  98027.0: 149,
  98597.0: 156,
  98012.0: 178,
  98366.0: 183,
  98001.0: 151,
  98144.0: 167,
  98110.0: 182},
 98012.0: {98122.0: 23,
  98109.0: 22,
  98027.0: 31,
  98597.0: 95,
  98903.0: 178,
  98366.0: 43,
  98001.0: 50,
  98144.0: 26,
  98110.0: 30},
 98366.0: {98122.0: 24,
  98109.0: 22,
  98027.0: 42,
  98597.0: 61,
  98903.0: 183,
  98012.0: 43,
  98001.0: 32,
  981

In [ ]:
# Create the Gurobi model
tsp_model = gb.Model("Traveling_Salesman_Problem")
tsp_model.Params.OutputFlag = 0  # Suppress Gurobi output

# Decision variables: x[i,j] = 1 if we travel from city i to city j
x = tsp_model.addVars(zipcodes, zipcodes, vtype=GRB.BINARY, name="x")


# Objective function: Minimize total travel distance
tsp_model.setObjective(
    gb.quicksum(graph[i][j] * x[i, j] for i in zipcodes for j in zipcodes if i != j), GRB.MINIMIZE
)


# Constraints
# 1. Each city must be visited exactly once
for city in zipcodes:
    tsp_model.addConstr(sum(x[city, j] for j in zipcodes if j != city) == 1, f"out_{city}")
    tsp_model.addConstr(sum(x[i, city] for i in zipcodes if i != city) == 1, f"in_{city}")


# # 2. Subtour elimination (Miller-Tucker-Zemlin formulation)
# u = tsp_model.addVars(zipcodes, vtype=GRB.INTEGER, lb=1, ub=len(zipcodes), name="u")

# for i in zipcodes:
#     for j in zipcodes:
#         if i != j:
#             tsp_model.addConstr(u[i] - u[j] + len(zipcodes) * x[i,j] <= len(zipcodes) - 1, f"subtour_{i}_{j}")


# Optimize the model
tsp_model.optimize()


if tsp_model.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Distance: {tsp_model.ObjVal}")

    # Print the optimal tour
    current_city = zipcodes[0]
    tour = [current_city]
    for _ in range(len(zipcodes) - 1):
        for next_city in zipcodes:
            if current_city != next_city and x[current_city, next_city].x > 0.5:
                tour.append(next_city)
                current_city = next_city
                break
    # Convert zipcodes in tour to strings before joining
    print(" -> ".join(map(str, tour))) # Changed line

else:
    print("No optimal solution found")

Optimal Solution Found:
Total Distance: 436.0
98122.0 -> 98144.0 -> 98122.0 -> 98144.0 -> 98122.0 -> 98144.0 -> 98122.0 -> 98144.0 -> 98122.0 -> 98144.0


# Extension 5: Multiple Maintenance Men

In [ ]:
num_salesmen = 2  # Number of maintenance men

# Create the Gurobi model
mtm_model = gb.Model("Multiple_Traveling_Salesmen_Problem")
mtm_model.Params.OutputFlag = 0  # Suppress Gurobi output

# Decision variables: x[k, i, j] = 1 if salesman k travels from city i to city j
x = mtm_model.addVars(range(num_salesmen), zipcodes, zipcodes, vtype=GRB.BINARY, name="x")

# Objective function: Minimize total travel distance for all salesmen
mtm_model.setObjective(
    gb.quicksum(graph[i][j] * x[k, i, j] for k in range(num_salesmen) for i in zipcodes for j in zipcodes if i != j),
    GRB.MINIMIZE
)

# Constraints
# 1. Each city must be visited exactly once by one salesman
for city in zipcodes:
    mtm_model.addConstr(sum(x[k, city, j] for k in range(num_salesmen) for j in zipcodes if j != city) == 1, f"out_{city}")
    mtm_model.addConstr(sum(x[k, i, city] for k in range(num_salesmen) for i in zipcodes if i != city) == 1, f"in_{city}")

# 2. Subtour elimination for each salesman
u = mtm_model.addVars(range(num_salesmen), zipcodes, vtype=GRB.INTEGER, lb=1, ub=len(zipcodes), name="u")

for k in range(num_salesmen):
    for i in zipcodes:
        for j in zipcodes:
            if i != j:
                mtm_model.addConstr(u[k, i] - u[k, j] + len(zipcodes) * x[k, i, j] <= len(zipcodes) - 1, f"subtour_{k}_{i}_{j}")


# Optimize the model
mtm_model.optimize()

if mtm_model.Status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    print(f"Total Distance: {mtm_model.ObjVal}")

    # Print the optimal tours for each salesman
for k in range(num_salesmen):
    print(f"\nSalesman {k + 1}:")
    current_city = zipcodes[0]  # Start from the first city (arbitrarily chosen)
    tour = [str(current_city)] # Initialize tour with string version of the first city
    visited = {current_city} # Keep track of visited cities
    for _ in range(len(zipcodes) - 1):
        for next_city in zipcodes:
            if current_city != next_city and x[k, current_city, next_city].x > 0.5 and next_city not in visited:
                tour.append(str(next_city)) # Append string version of next_city to tour
                visited.add(next_city)
                current_city = next_city
                break

    print(" -> ".join(tour)) # Now tour contains strings, join should work

else:
    print("No optimal solution found")

NameError: name 'gb' is not defined

# HARDCODING Distances Instead of using CSV

In [ ]:
# prompt: pick 10 random zipcodes as nodes, make up some distances randomly and set up a traveling maintance man problem

import random
import pandas as pd

# Generate 10 random zip codes (replace with actual zip codes if needed)
zipcodes = [random.randint(10000, 99999) for _ in range(10)]

# Create a dictionary to store distances between zip codes
distances = {}
for i in range(len(zipcodes)):
    distances[zipcodes[i]] = {}
    for j in range(len(zipcodes)):
        if i != j:
            distances[zipcodes[i]][zipcodes[j]] = random.randint(10, 100)  # Random distance between 10 and 100

# Create a pandas DataFrame from the distances data
distance_data = []
for zip1 in distances:
  for zip2 in distances[zip1]:
    distance_data.append({'PostalCode1': zip1, 'PostalCode2': zip2, 'Distance': distances[zip1][zip2]})
distance_df = pd.DataFrame(distance_data)

# Save the distance data to a CSV file
distance_df.to_csv('distance.csv', index=False)

In [ ]:
# Extract 10 unique zipcodes from the distance file
zipcodes = list(distance_data['PostalCode1'].unique()[:10])

# Create a list to store zipcode-distance pairs
zipcodes

[98122.0,
 98109.0,
 98027.0,
 98597.0,
 98903.0,
 98012.0,
 98366.0,
 98001.0,
 98144.0,
 98110.0]

In [ ]:
# List of postal codes
postal_codes = [
    98122.0, 98109.0, 98027.0, 98597.0, 98903.0,
    98012.0, 98366.0, 98001.0, 98144.0, 98110.0
]

# Generate random symmetric distances between nodes
np.random.seed(42)
n = len(postal_codes)
distances = np.random.randint(10, 500, size=(n, n))
distances = (distances + distances.T) // 2  # Symmetry
np.fill_diagonal(distances, 0)  # Zero diagonal (distance to itself)

# Convert to DataFrame for clarity
distance_data = pd.DataFrame(
    [(postal_codes[i], postal_codes[j], distances[i, j]) for i in range(n) for j in range(n) if i != j],
    columns=["PostalCode1", "PostalCode2", "Distance"]
)

# Display the data
print(distance_data.head())


   PostalCode1  PostalCode2  Distance
0      98122.0      98109.0       460
1      98122.0      98027.0       258
2      98122.0      98597.0       225
3      98122.0      98903.0       147
4      98122.0      98012.0       227


In [ ]:
# Create a Gurobi model
model = Model("TSP")

# Define nodes and distances
nodes = postal_codes
distances_dict = {(row.PostalCode1, row.PostalCode2): row.Distance for _, row in distance_data.iterrows()}

# Decision variables: travel[i, j] = 1 if traveling from i to j
travel = model.addVars(nodes, nodes, vtype=GRB.BINARY, name="Travel")

# Objective: Minimize total travel distance
model.setObjective(
    quicksum(distances_dict[i, j] * travel[i, j] for i in nodes for j in nodes if i != j),
    GRB.MINIMIZE
)

# Constraints:
# 1. Each node must have exactly one incoming and one outgoing edge
model.addConstrs((quicksum(travel[i, j] for j in nodes if i != j) == 1 for i in nodes), name="Outflow")
model.addConstrs((quicksum(travel[j, i] for j in nodes if i != j) == 1 for i in nodes), name="Inflow")

# # 2. Subtour Elimination (MTZ Formulation)
# u = model.addVars(nodes, vtype=GRB.INTEGER, lb=0, ub=len(nodes) - 1, name="u")  # Auxiliary variables
# for i in nodes:
#     for j in nodes:
#         if i != j:
#             model.addConstr(u[i] - u[j] + len(nodes) * travel[i, j] <= len(nodes) - 1, name=f"Subtour_{i}_{j}")


# Solve the model
model.optimize()

# Display the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal Tour:")
    for i in nodes:
        for j in nodes:
            if i != j and travel[i, j].x > 0.5:  # Check if the edge is part of the solution
                print(f"Travel from {i} to {j}")
    print(f"\nTotal Distance: {model.objVal}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 20 rows, 100 columns and 180 nonzeros
Model fingerprint: 0x5012cc6d
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2521.0000000
Presolve removed 0 rows and 10 columns
Presolve time: 0.00s
Presolved: 20 rows, 90 columns, 180 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 1.452000e+03, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0 

## Enviromental Grant Extension

In [ ]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd

# Load data
ev_data = pd.read_csv("/WA_Electric_Vehicle_Population_Data.csv", dtype={'Postal Code': str})
distance_data = pd.read_csv("/distance.csv", dtype={'PostalCode1': str, 'PostalCode2': str})

# Define coverage function
def calculate_coverage(row, average_ranges, theta):
    postal1 = row['PostalCode1']
    postal2 = row['PostalCode2']
    distance = row['Distance']
    coverage1 = 1 if (theta * average_ranges.get(postal1, 0) >= distance) else 0
    coverage2 = 1 if (theta * average_ranges.get(postal2, 0) >= distance) else 0
    return pd.Series([coverage1, coverage2])

def calculate_C(distance_data):
    C = {}
    for _, row in distance_data.iterrows():
        C[(row['PostalCode1'], row['PostalCode2'])] = row['Coverage1']
        C[(row['PostalCode2'], row['PostalCode1'])] = row['Coverage2']
    postal_codes = set(distance_data['PostalCode1']).union(distance_data['PostalCode2'])
    for postal_code in postal_codes:
        C[(postal_code, postal_code)] = 1
    return C

# Prepare data for the model
EV_count = ev_data['Postal Code'].value_counts().to_dict()
average_range = ev_data.groupby('Postal Code')['Electric Range'].mean().to_dict()
EV = {key: EV_count[key] * average_range[key] for key in EV_count if key in average_range}

distance_data[['Coverage1', 'Coverage2']] = distance_data.apply(
    calculate_coverage, axis=1, average_ranges=average_range, theta=0.2
)
C = calculate_C(distance_data)

# Parameters
charging_speed = {2: 15, 3: 210}
environmental_impact = {2: 0.000027405, 3: 0.0002945}
cost_per_level2_charger = 8250
cost_per_level3_charger = 65000
grant_amount = 50000
impact_threshold = 10
H = 10  # Available hours per day

# Initialize model
model = gb.Model("EV_Charging_Optimization_With_Grant")

# Decision Variables
x2 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level2Chargers")
x3 = model.addVars(EV.keys(), lb=0, vtype=GRB.INTEGER, name="Level3Chargers")
grant = model.addVar(vtype=GRB.BINARY, name="Grant")

# Environmental impact calculation
total_impact = gb.quicksum(
    x2[i] * environmental_impact[2] + x3[i] * environmental_impact[3] for i in EV
)

# Apply the grant conditionally
model.addConstr(
    grant * grant_amount <= (impact_threshold - total_impact) * 1000000,
    "GrantCondition"
)

# Total cost calculation
total_cost = gb.quicksum(
    x2[i] * cost_per_level2_charger + x3[i] * cost_per_level3_charger for i in EV
) - grant * grant_amount

# Objective: Minimize total cost
model.setObjective(total_cost, GRB.MINIMIZE)

# Demand satisfaction constraints
model.addConstrs(
    (x2[i] * charging_speed[2] * H + x3[i] * charging_speed[3] * H >= EV[i]
     for i in EV),
    "DemandSatisfaction"
)

# Solve the model
model.optimize()

# Output results
if model.Status == GRB.OPTIMAL:
    print(f"Optimal Total Cost: {total_cost.getValue()}")
    print(f"Total Environmental Impact: {total_impact.getValue()}")
    print(f"Grant Applied: {grant.X}")
    for i in EV:
        print(f"Location {i}: Level 2 Chargers = {x2[i].X}, Level 3 Chargers = {x3[i].X}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 48 physical cores, 96 logical processors, using up to 32 threads

Optimize a model with 520 rows, 1039 columns and 2077 nonzeros
Model fingerprint: 0x4a5f500d
Variable types: 0 continuous, 1039 integer (1 binary)
Coefficient statistics:
  Matrix range     [3e+01, 5e+04]
  Objective range  [8e+03, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 1e+07]
Found heuristic solution: objective 4.690078e+08
Presolve removed 519 rows and 1037 columns
Presolve time: 0.09s
Presolved: 1 rows, 2 columns, 2 nonzeros
Found heuristic solution: objective 3.312175e+08
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 3.311571e+08, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf